In [1]:
import keras
from nltk.tokenize import word_tokenize
import re
import os

Using TensorFlow backend.


In [2]:
from konlpy.tag import Kkma, Twitter
import soynlp

In [3]:
data_path = "./data/Memory_End_to-End_Network_Project/bAbi-tasks/QA_bAbI_tasks/ko/"

In [4]:
f = open(data_path + "qa1_single-supporting-fact_train.txt")
docs = f.readlines()

In [7]:
data[0]

'1 마리아가 화장실로 갔다.\n'

In [8]:
word2idx = {}
max_words = 0
max_sentences = 0

In [9]:
def read_data(fname, word2idx, max_words, max_sentences):
    stories = dict()
    questions = dict()
    
    if len(word2idx) == 0:
        word2idx['<null>'] = 0
        
    if os.path.isfile(fname):
        with open(fname) as f:
            lines = f.readlines()
    else:
        raise Exception("[!] Data {file} not found".format(file=fname))
        
    for line in lines:
        line = line.strip()  # delete \n
        nid, line = line.split(' ', 1)  # seperate number and sentences
        line = line.lower()
        nid = int(nid)
        
        # whether story begins
        if nid == 1:
            story_ind = len(stories)
            sentence_ind = 0
            stories[story_ind] = []
        
        if '\t' in line:
            is_question = True
            question_ind = len(questions)
            questions[question_ind] = {'question': [], 
                                       'answer': [], 
                                       'story_index': story_ind, 
                                       'sentence_index': sentence_ind}
        else:
            is_question = False
            sentence_ind = len(stories[story_ind])
        
        # sentence parsing
        sentence_list = []
        if not is_question:
            words_list, word2idx = tokennvocab(line, word2idx, is_question)
            for w in words_list:
                if w != '.':
                    sentence_list.append(w)
                else:
                    stories[story_ind].append(sentence_list)
                    break
        else:
            words_list, answer, supporting, words2idx = tokennvocab(line, word2idx, is_question)
            for w in words_list:
                if w != '?':
                    sentence_list.append(w)
                else:
                    questions[question_ind]['question'].extend(sentence_list)
                    questions[question_ind]['answer'].append(answer)
                    break
        
        # Update max_sentences
        max_sentences = max(max_sentences, sentence_ind)

        
            # Convert the words into indices
    for idx, context in stories.items():
        for i in range(len(context)):
            temp = list(map(word2idx.get, context[i]))
            context[i] = temp
    
    for idx, value in questions.items():
        temp1 = list(map(word2idx.get, value['question']))
        temp2 = list(map(word2idx.get, value['answer']))
        
        value['question'] = temp1
        value['answer'] = temp2
        
        
        
    return stories, questions, max_words, max_sentences

In [10]:
def tokennvocab(line, word2idx, is_question=True):
    """
    tokenizing + add words to dictionary
    
    """
    words_list = word_tokenize(line)
    
    if not is_question:
        words = words_list.copy()
        words.remove('.')
        for w in words:
            if w not in word2idx:
                word2idx[w] = len(word2idx)   
        return words_list, word2idx

    else:
        supporting = words_list[-1]
        answer = words_list[-2]
        words_list = words_list[:-1]
        words = words_list.copy()
        words.remove('?')
        for w in words:
            if w not in word2idx:
                word2idx[w] = len(word2idx)
        return words_list, answer, supporting, word2idx

In [12]:
stories, questions, max_words, max_sentences = read_data(data_path + "qa1_single-supporting-fact_train.txt", word2idx, max_words, max_sentences)

In [5]:
doc = docs[1].strip()

In [6]:
doc

'2 요한은 복도로 갔다.'

In [7]:
nid, doc = doc.split(' ', 1)

In [8]:
doc

'요한은 복도로 갔다.'

In [78]:
word2idx

{'<null>': 0,
 '?': 9,
 '갔다': 3,
 '그': 36,
 '다니엘은': 11,
 '다니엘이': 26,
 '다시': 30,
 '동산에': 20,
 '동산으로': 27,
 '마리아가': 1,
 '마리아는': 6,
 '복도': 14,
 '복도로': 5,
 '사무실': 16,
 '사무실로': 15,
 '사무실에': 23,
 '산드라가': 12,
 '산드라는': 18,
 '어디에': 7,
 '왔어': 35,
 '요한은': 4,
 '이사': 34,
 '이사갔습니다': 31,
 '이사왔다': 32,
 '있습니까': 8,
 '정원': 28,
 '정원에': 33,
 '정원으로': 13,
 '주방': 25,
 '주방에': 29,
 '주방으로': 24,
 '침실': 22,
 '침실로': 17,
 '침실에': 21,
 '화장실': 10,
 '화장실로': 2,
 '화장실에': 19}

In [ ]:
def cohension(w):
    return

In [81]:
docs = data

In [83]:
data[0].split(' ', 1)

['1', '마리아가 화장실로 갔다.\n']

In [9]:
from collections import defaultdict
count = defaultdict(lambda: 0)

for doc in docs:
    _, doc = doc.split(' ', 1)
    if '\t' in doc:
        #is_question = True
        doc = word_tokenize(doc)[:-1]
    else:
        #is_question = False
        doc = word_tokenize(doc)
    for word in doc:
        n = len(word)
        for e in range(1, n+1):
            count[word[:e]] += 1

In [10]:
count

defaultdict(<function __main__.<lambda>>,
            {'.': 2000,
             '?': 2000,
             '갔': 1986,
             '갔다': 1986,
             '그': 1,
             '다': 805,
             '다니': 743,
             '다니엘': 743,
             '다니엘은': 582,
             '다니엘이': 161,
             '다시': 62,
             '동': 93,
             '동산': 93,
             '동산에': 31,
             '동산으': 62,
             '동산으로': 62,
             '마': 742,
             '마리': 742,
             '마리아': 742,
             '마리아가': 160,
             '마리아는': 582,
             '복': 526,
             '복도': 526,
             '복도로': 351,
             '사': 500,
             '사무': 500,
             '사무실': 500,
             '사무실로': 259,
             '사무실에': 75,
             '산': 761,
             '산드': 761,
             '산드라': 761,
             '산드라가': 349,
             '산드라는': 412,
             '어': 1000,
             '어디': 1000,
             '어디에': 1000,
             '왔': 3,
             '왔어': 3,
             '

In [13]:
def cohension(w):
    return pow(count[w]/count[w[0]], 1/(len(w)-1))

In [20]:
word = '주방으로'

In [21]:
n = len(word)

In [22]:
for e in range(2, n+1):
    w = word[:e]
    f = count[w]
    s = cohension(w)
    print('{:6}, f={}, s={:.2}'.format(w, f, s))

주방    , f=511, s=1.0
주방으   , f=252, s=0.7
주방으로  , f=252, s=0.79


In [ ]:
def tokenizer(w):
    n = len(w)
    if n <= 2:
        return (w, '')
    tokens = []
    for e in range(2, n+1):
        